In [123]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


class MLP(nn.Module):
    def __init__(self) -> None:
        super(MLP,self).__init__()

        self.w00 = nn.Parameter(torch.tensor(0.22),requires_grad=True)
        self.w01 = nn.Parameter(torch.tensor(0.05),requires_grad=True)
        self.b0 = nn.Parameter(torch.tensor(-0.05),requires_grad=True)

        self.w100 = nn.Parameter(torch.tensor(0.01),requires_grad=True)
        self.w101 = nn.Parameter(torch.tensor(0.01),requires_grad=True)
        self.w110 = nn.Parameter(torch.tensor(0.01),requires_grad=True)
        self.w111 = nn.Parameter(torch.tensor(0.01),requires_grad=True)
        self.b1 = nn.Parameter(torch.tensor(0.05),requires_grad=True)

        self.w20 = nn.Parameter(torch.tensor(0.05),requires_grad=True)
        self.w21 = nn.Parameter(torch.tensor(0.05),requires_grad=True)
        self.b2 = nn.Parameter(torch.tensor(0.05),requires_grad=True)



    def forward(self,x):
        u10 = F.relu(x*self.w00+self.b0)
        u11 = F.relu(x*self.w01+self.b0)

        u20 = F.relu(u10*self.w100+u11*self.w110+self.b1)
        u21 = F.relu(u10*self.w101+u11*self.w111+self.b1)

        final = F.relu(u20*self.w20+u21*self.w21+self.b2)

        return final

In [153]:
######## Simulate Y = 3*X^2

X = torch.unsqueeze(torch.linspace(-100, 100, 10000), dim=1)
Y = 3*X**2

In [154]:
class PolyDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, X, Y, transform=None):
        """
        Args:
            X: list of x variables
            Y: list of y variables
        """
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.index_select(self.X, 0, torch.tensor(idx))
        y = torch.index_select(self.Y, 0, torch.tensor(idx))
        sample = (x,y)

        if self.transform:
            sample = self.transform(sample)

        return sample

In [155]:
dataset = PolyDataset(X,Y)

In [156]:
trainloader = torch.utils.data.DataLoader(dataset,batch_size=1024,shuffle=True)

In [157]:
model = MLP()
optimizer = torch.optim.Adam(model.parameters())

criterion = nn.MSELoss()

In [ ]:
for epoch in range(100):
    
    for batch in trainloader:
        optimizer.zero_grad()
        x,y = batch
        preds = model(x)
        loss = criterion(y,preds).sqrt()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Training Loss: ",loss)
    

In [60]:
model(torch.tensor([1.0,1.1]))

tensor([0., 0.], grad_fn=<ReluBackward0>)

In [159]:
for batch in trainloader:
    x,y = batch
    print(x)
    break

tensor([[[-70.4370]],

        [[ 11.2311]],

        [[ 85.5386]],

        ...,

        [[ 66.6967]],

        [[ 90.1190]],

        [[ 21.5522]]])


In [160]:
model(torch.tensor([2]))

tensor([10.0536], grad_fn=<ReluBackward0>)

In [161]:
class MLPTorch(nn.Module):
    def __init__(self) -> None:
        super(MLPTorch,self).__init__()

        self.linear1 = nn.Linear(1,32)
        self.linear2 = nn.Linear(32,64)
        self.linear3 = nn.Linear(64,64)
        self.linear4 = nn.Linear(64,32)
        self.linear5 = nn.Linear(32,1)

    def forward(self,x):
        x = self.linear1(x).relu()
        x = self.linear2(x).relu()
        x = self.linear3(x).relu()
        x = self.linear4(x).relu()
        x = self.linear5(x)

        return x

In [162]:
model = MLPTorch()
optimizer = torch.optim.Adam(model.parameters())

criterion = nn.MSELoss()
trainloader = torch.utils.data.DataLoader(dataset,batch_size=1024,shuffle=True)
for epoch in range(1000):
    for batch in trainloader:
        x,y = batch
        preds = model(x)
        loss = criterion(y,preds).sqrt()
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()
    print(f"Epoch {epoch+1}, Training Loss: ",loss)
    

Epoch 1, Training Loss:  tensor(13597.2217, grad_fn=<SqrtBackward0>)
Epoch 2, Training Loss:  tensor(13760.2881, grad_fn=<SqrtBackward0>)
Epoch 3, Training Loss:  tensor(13596.1904, grad_fn=<SqrtBackward0>)
Epoch 4, Training Loss:  tensor(13204.8057, grad_fn=<SqrtBackward0>)
Epoch 5, Training Loss:  tensor(13410.2344, grad_fn=<SqrtBackward0>)
Epoch 6, Training Loss:  tensor(12893.1123, grad_fn=<SqrtBackward0>)
Epoch 7, Training Loss:  tensor(12342.9854, grad_fn=<SqrtBackward0>)
Epoch 8, Training Loss:  tensor(10794.9980, grad_fn=<SqrtBackward0>)
Epoch 9, Training Loss:  tensor(8833.3887, grad_fn=<SqrtBackward0>)
Epoch 10, Training Loss:  tensor(5679.3022, grad_fn=<SqrtBackward0>)
Epoch 11, Training Loss:  tensor(4044.6589, grad_fn=<SqrtBackward0>)
Epoch 12, Training Loss:  tensor(3470.2327, grad_fn=<SqrtBackward0>)
Epoch 13, Training Loss:  tensor(3474.2234, grad_fn=<SqrtBackward0>)
Epoch 14, Training Loss:  tensor(3428.6294, grad_fn=<SqrtBackward0>)
Epoch 15, Training Loss:  tensor(34

In [163]:
model(torch.tensor([3.0]))

tensor([27.6593], grad_fn=<AddBackward0>)

In [164]:
torch.save(model.state_dict(), './polynomial.pt')